In [1]:
import pyaudio
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops import gen_audio_ops as audio_ops
from datetime import datetime

model = keras.models.load_model("fully_trained.model")

FORMAT = pyaudio.paFloat32
RATE = 16000
CHANNELS = 1
NOFFRAMES = 8000

audio = pyaudio.PyAudio()

info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
    if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ",
              audio.get_device_info_by_host_api_device_index(0, i).get('name'))


samples = np.zeros((8000))


def callback(input_data, frame_count, time_info, flags):
    global samples
    # print("Got audio " + str(frame_count))
    new_samples = np.frombuffer(input_data, np.float32)
    samples = np.concatenate((samples, new_samples))
    samples = samples[-16000:]

    if len(samples) == 16000:
        start = time.perf_counter()
        # normalise the samples
        normalised = samples - np.mean(samples)
        max = np.max(normalised)
        if max > 0:
            normalised = normalised / max

        # create the spectrogram
        spectrogram = audio_ops.audio_spectrogram(
            np.reshape(normalised, (16000, 1)),
            window_size=320,
            stride=160, 
            magnitude_squared=True)
        # reduce the number of frequency bins in our spectrogram to a more sensible level
        spectrogram = tf.nn.pool(
            input=tf.expand_dims(spectrogram, -1),
            window_shape=[1, 6],
            strides=[1, 6],
            pooling_type='AVG',
            padding='SAME')
        # remove the first 1 index
        spectrogram = tf.squeeze(spectrogram, axis=0)
        spectrogram = np.log10(spectrogram + 1e-6)
        prediction = model.predict(np.reshape(spectrogram, (1, 99, 43, 1)))        
        
        index = 0
        max_value = 0
        for i in range(4):
            if prediction[0][i] > max_value:
                max_value = prediction[0][i]
                index = i
        
        dict_predict = {0 : 'backward', 1: 'right', 2:'down', 3:'invalid'}        
        if max_value > 0.95 and index != 3:
            print(
                f"{datetime.now().time()} - {dict_predict[index]}, and score: {max_value}")
        
        end = time.perf_counter()
        # print((end-start)*1000)

    return input_data, pyaudio.paContinue

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Microphone Array (טכנולוגיית In


In [2]:
stream = audio.open(
    input_device_index=0,
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    stream_callback=callback,
    frames_per_buffer=NOFFRAMES)

stream.start_stream()
print("speak")

# wait for stream to finish (5)
while stream.is_active() and time.time:
    time.sleep(0.1)

speak
18:59:25.140162 - down, and score: 0.9933440685272217
18:59:40.642372 - backward, and score: 0.9989527463912964
18:59:41.143186 - backward, and score: 0.9988227486610413
18:59:49.237519 - backward, and score: 0.9863978028297424
18:59:56.141917 - right, and score: 0.9874160885810852
19:00:08.142313 - backward, and score: 0.9566164612770081
19:00:11.132846 - right, and score: 0.9985013008117676
19:00:11.642252 - right, and score: 0.9999924898147583
19:00:13.139917 - right, and score: 0.9999488592147827
19:00:23.151297 - down, and score: 0.9746382236480713
19:00:28.140886 - backward, and score: 0.9623441696166992
19:00:30.148737 - right, and score: 0.9973684549331665
19:00:44.639720 - backward, and score: 0.9685735106468201
19:01:02.639582 - backward, and score: 0.9915940761566162
19:01:23.129948 - down, and score: 0.9586398005485535
19:01:32.647412 - down, and score: 0.9609899520874023
19:02:03.139293 - backward, and score: 0.9804110527038574
19:02:46.636075 - backward, and score: 

In [ ]:
# stream.stop_stream()
# stream.close()
# p.terminate()
# print('done')
# plt.plot(decoded)
# plt.show()